## ICU Chartevents

The code below keeps only the chartevents rows from patients with disease we have declare to the csv file and merge them with the file o03_icu_first_stay.csv

Export file -> CSV\Exports\o04_icu_chartevent.csv

------------------

I apply the chunk command because the file contains 313,645,063 lines which causes the computer to run out of memory.

I have set the size of the chunksize variable it will read to 100,000 rows. I set it according to the available computer memory.

In [ ]:
import gc
import gzip
import pandas as pd

#Read the first_rows_df CSV file
first_stay_df = pd.read_csv('CSV\Exports\o03_icu_first_stay.csv')

# Read the chartevents CSV file in chunks
chunksize = 100000  # Number of rows to read in each chunk
chartevents_df = pd.read_csv(r"..\Datasets\mimic-iv-2_2\icu\chartevents.csv.gz", chunksize=chunksize)

# Create a list to store the processed chunks
processed_chunks = []

# Iterate over the chunks and process them
for chunk in chartevents_df:
    # Merge the first_stay_df and chartevents_df DataFrames on the stay_id column
    merged_df = pd.merge(first_stay_df, chunk, on='stay_id')

    # Add the processed chunk to the list
    processed_chunks.append(merged_df)

# Combine the processed chunks into a single DataFrame
chartevent_df = pd.concat(processed_chunks)

# Drop the columns subject_id_y, hadm_id_y
chartevent_df = chartevent_df.drop(columns=['subject_id_y', 'hadm_id_y'])

# Rename the columns subject_id_x to subject_id and hamd_id_x to hadm_id
chartevent_df = chartevent_df.rename(columns={'subject_id_x': 'subject_id', 'hadm_id_x': 'hadm_id'})

# Keeping the rows that contain values in the valuenum
chartevent_df = chartevent_df[chartevent_df['valuenum'].notna()]

# Save chartevent dataset to a CSV file
chartevent_df.to_csv('CSV\Exports\o04_icu_chartevent.csv', index=False)

# Free RAM
gc.collect()